In [1]:
eyes_api_key = '';

from traitlets.config.manager import BaseJSONConfigManager
path = "C:\\Users\\stanrogo\\Anaconda3\\etc\\jupyter\\nbconfig"
cm = BaseJSONConfigManager(config_dir=path)
cm.update('livereveal', {
              'theme': 'Simple',
              'transition': 'slide',
              'scroll': True
})

{'scroll': True,
 'start_slideshow_at': 'selected',
 'theme': 'Simple',
 'transition': 'slide'}

# Integrating Visual Regression Tests in Your Development Life Cycle
## By Stanley Clark

<img src="images/studyportals.png" style="margin: 0 auto; padding: 6rem 0 0 0;"/>

# About Me

* Stanley Clark
* Front-end Web Engineer @ StudyPortals (1.5 years)
* Studying Computer Science & Engineering M.Sc.
* Search Product Team
* @stanrogo

<img src="images/me.jpg" alt="Drawing" style="float:right;margin-top:-20vh;width: 200px;"/>

# Background Info

* StudyPortals - Global Study Choice Platform
* 5 scrum teams
* Different products, shared codebase
* PHP rendered templates with traditional JS UI
* Continuous integration present
* Continuous delivery is being implemented

# The Plan For Today

* Regression Testing Motivation
* Process Implementation At StudyPortals
* Thinking Outside Of The Code

And a few code examples along the way!
Slides are posted at https://stanrogo.com/vrt

# 1. Motivation

## human.capabilities !== computer.capabilities

<img src="images/spot_the_difference.jpg"/>

## Obvious Changes Go Un-noticed

Example: Derren Brown's "Person Swap"
<img src="images/derren_brown.png"/>

## What Are Regression Tests?

>"A type of software testing which verifies that software which was previously developed and tested still performs correctly after it was changed or interfaced with other software"

**Visual** Regression Tests in essence ensure that you don't screw up your site by accident.

Tests take screenshots of areas or pages of your site, compare those against a baseline and fail when things change.

## Why Visual Regression Testing At StudyPortals?

### Unit Testing Didn't Invoke Enough Value
* Templates generated via php -> computation also
* Few modules with exclusively non-DOM operations
* Doesn't check for usability -> Somebody could have added body{opacity: 0} and you wouldn't know

### Visual Bugs From Cross Team Work Released Too Often

>But the tutorial told me this would be easy...

# 2. Implementation In Our Organisation

## 2.1. Desired Development Workflow

1. Engineer makes some changes to the code and pushes them to a feature branch.
2. Regression tests are run.
3. Status is reported.
4. Engineer accepts or rejects the new baseline.
5. Engineer fixes issues if needed and repeats steps 1 - 3.
6. Code is good to go and can be merged into the rest of the code base.

## 2.2. The Process - 1st Iteration

1. Engineer makes some changes to the code and pushes them to a feature branch.
2. Branch is merged.
3. Regression tests are run automatically.
4. Status is reported in Slack.
5. Engineer accepts are rejects the new baseline.
6. Engineer fixes issues if needed.

### 2.2.1. Make Changes To Code And Push

Standard GitHub repository used
<img src="images/github.png" alt="Drawing" style="float:right;width: 200px;"/>

### 2.2.2. Merge Branch & Run Regression Test

* Using existing jenkins and docker infrastructure.
* Detect a merge has been done and run tasks on a dedicated regression testing server.
<img src="images/docker-jenkins.png" alt="Drawing" style="float:right;width: 200px;"/>

#### 2.2.2.1. Tooling Used

<div style="display:flex;margin: 0 auto; padding: 2rem 0;">
    <figure style="width: 25%; padding: 0 5rem 0 0; text-align: center;">
        <img src="images/ubuntu.svg"/>
        <figcaption style="line-height:1;">Ubuntu Server</figcaption>
    </figure>
    <figure style="width: 25%; padding: 0 5rem 0 0; text-align: center;">
        <img src="images/phantomjs.png"/>
        <figcaption style="line-height:1;">PhantomJS (and later Firefox) in headless mode.</figcaption>
    </figure>
    <figure style="width: 25%;padding: 0 5rem 0 0; text-align: center;">
        <img src="images/python.png"/>
        <figcaption style="line-height:1;">Python Selenium Bindings</figcaption>
    </figure>
    <figure style="width: 25%;padding: 0 5rem 0 0; text-align: center;">
        <img src="images/applitools.png"/>
        <figcaption style="line-height:1;">Applitools Eyes (By Attlasian)</figcaption>
    </figure>
</div>

### 2.2.2.2. Applitools Eyes

* Paid service
* Central dashboard to view all differences in one place
* All image processing done on their servers -> speed
* Support from JS, Python, Java, Ruby, as well as non-code direct browser support

#### 2.2.2.3. Simple VRT test demo

In [9]:
from selenium import webdriver
from applitools.eyes import Eyes
import time

eyes = Eyes()
eyes.api_key = eyes_api_key
eyes.force_full_page_screenshot = False
eyes.is_disabled = True

#### 2.2.2.3. Simple VRT test demo

In [10]:
    driver = webdriver.Firefox()
    eyes.open(driver=driver, app_name='Masters Portal', test_name='Form validation',
              viewport_size={'width': 1600, 'height': 800})
    driver.get('https://www.mastersportal.eu/studies/5/')
    register_button = driver.find_element_by_id("RegisterButton").click()
    login_placeholder = driver.find_element_by_id("LoginPlaceholderContainer")
    eyes.check_region_by_element(login_placeholder, 'Study #5 register view')
    time.sleep(2)    
    submit = driver.find_element_by_id("js-RegistrationButton")
    username = driver.find_element_by_id("authSignUpName")
    email = driver.find_element_by_id("authSignUpEmail")
    password = driver.find_element_by_id("authSignUpPassword")
    username.send_keys("John Doe")
    email.send_keys("test@email.nl")
    password.send_keys("testpassword")
    submit.click()
    eyes.check_region_by_element(login_placeholder, 'Study #5 register view - submit form')
    time.sleep(2)
    login_view = driver.find_element_by_css_selector("li[data-action=\"in\"]")
    login_view.click()
    driver.find_element_by_css_selector("li[data-action=\"in\"]")
    eyes.check_region_by_element(login_placeholder, 'Study #5 login view')
    eyes.close()

### 2.2.3. Output is reported in Slack

Use the slack API to send messages to a specific channel, and tag the user while doing it.
<img src="images/slack.svg" alt="Drawing" style="float:right;width: 200px;"/>

### 2.2.4. Developer Accepts or rejects baselines

A few things to take into account:

* Is the failing test caused by another team?
* Is the passing test caused by your changes not actually working?
* If something has been failing due to its dynamicity, perhaps rethink the test.

#### 2.2.4.1. When Changes Are OK
<img src="images/change-ok.png" alt="Drawing"/>

#### 2.2.4.2. When  Changes Are Not OK
<img src="images/change-bad.png" alt="Drawing"/>

#### 2.2.4.3. Defects Caught

* Description on search results was no longer being trimmed
* Search box in the nav bar was displaying incorrectly on some devices after introduction of Post CSS
* Testing login details not working (impedes testing on actual devices)
* Currency not displaying correctly due to JavaScript breaking
* And more...

#### 2.2.4.4. Not Quite Good Enough

## 2.3. The Process - 2nd Iteration

1. Engineer makes some changes to the code and pushes them to a feature branch.
2. **Engineer makes n commits and creates a pull request.**
3. **When ready to merge**, mark as "to test" and regression tests are run.
4. Status is reported **in GitHub**.
4. Engineer accepts or rejects the new baseline.
5. If needed, engineer fixes issues and repeats steps 1 - 4.
6. Code is good to go and can be merged into the rest of the code base.

### 2.3.1. When ready to merge, mark as "to test" and regression tests are run
<img src="images/test-label.png" alt="Drawing"/>

### 2.3.2. Status is reported in GitHub
<img src="images/running-test2.png" alt="Drawing"/>

# 3. Thinking Outside Of The Code

## 3.1. Cross Browser Testing

* Currently only running tests on Firefox
* Future adds Chrome to the mix
* Testing Multiple browsers adds review time and increases price
* IE and Safari testing hard to achieve on Ubuntu

## 3.2. Strictness Levels
VRT is not a one-size fits all solution. There are different solutions for different needs:

* Exact Match
* Layout
* Content

## 3.3. Keeping Scope Of Written Tests

* What exactly do you tests?
* How to make sure tests overlap as little as possible?
* Mindset of testing the visuals, rather than the functionality.
* If a targetting class changes, or something breaks, action needs to be taken by someone?

## 3.4. It's not perfect...

# 4. Tooling For The Job

Many solutions available out there. We used applitools in this demo.
You can use other tooling based on your requirements. Some notable ones are:

* BBC News Wraith https://github.com/BBC-News/wraith
* Backstop JS to locally integrate with Gulp or Grunt https://github.com/garris/BackstopJS
* Percy https://percy.io/

# 5. #TLDR;

* Visual Regression Testing is more accurate than manual testing by humans
* Unit testing is neither the only solution nor sometimes the most valuable
* Infrastructure is easy to set up in your existing continuous integration environment
* Plenty of visual diff solutions available, chooseone which suits you best
* Non code aspects are the trickiest to get right

To start playing, just install selenium for python and get a free applitools account!

<center> <h1>Questions?</h1> </center>
<img src="images/studyportals.png" style="margin: 0 auto; padding: 6rem 0 0 0;"/>